# Caret-Separated Text

Caret-Separated Text (or CST) is a key-value pair format represented by numbers as keys and the value is the string enclosed between carets (^) that contains the translation. Any text which is not enclosed with carets is considered a comment and ignored.

## CST.NET

CST.NET uses .NET's built-in indexing extension function to accomplish locating of each respective key. As a consequence, it does not matter what you use for keys. I added an additional normalizion to the pipeline that converts the document's line endings to the system's, in order to prevent crashes.

In [1]:
using System.IO;
using System.Text;

In [1]:
public static class CST
{
    public static string Parse(string cst, int key, params string[] args)
    {
        var entries = NormalizeEntries(cst);
        return GetEntry(entries, $"{key}", args);
    }

    public static string Parse(string cst, string key, params string[] args)
    {
        var entries = NormalizeEntries(cst);
        return GetEntry(entries, key, args);
    }

    static IEnumerable<string> NormalizeEntries(string cst)
    {
        var lineBreaks = new string[] { "^\u000A", "^\u000D", "^\u000A" };

        foreach (var line in lineBreaks)
        {
            var eol = Environment.NewLine; // System's line break

            // If the new line matches the system's, do nothing
            if (line.Contains(eol))
                continue;

            cst.Replace(line, eol);
        }

        return cst.Split(lineBreaks, StringSplitOptions.RemoveEmptyEntries);

    }

    static string ArgumentParser(string content, string[] args)
    {
        var sb = new StringBuilder();

        for (var i = 0; i < content.Length; i++)
        {
            var curArgs = content.Substring(i, 1);
            var argsCounter = 0;

            if (curArgs.Contains("%"))
            {
                if (argsCounter < args.Length)
                {
                    sb.Append(curArgs.Replace("%", args[argsCounter]));
                    i++;
                }
            }
            else
                sb.Append(curArgs);
        }

        return sb.ToString();
    }

    static string GetEntry(IEnumerable<string> entries, string key,
        params string[] args)
    {
        var translation = "[ENTRY NOT FOUND]";

        // Search through array
        foreach (var entry in entries)
        {
            // Locate index, trim carets and return translation
            if (!entry.StartsWith(key))
                continue;
                
            const char caret = '^';

            var startIndex = entry.IndexOf(caret.ToString(),
                StringComparison.OrdinalIgnoreCase);

            var line = entry.Substring(startIndex);

            var content = line.TrimStart(caret).TrimEnd(caret);

            if (args.Length > 0)
                translation = ArgumentParser(content, args);
            else
                translation = content;
        }

        return translation;
    }
}

In [1]:
var v1Path = Path.Combine(Environment.CurrentDirectory, "data", "v1.cst");
var v1File = File.ReadAllText(v1Path);
var singleLine = CST.Parse(v1File, 1, "FENNEC");
var multiLine = CST.Parse(v1File, 2);
Console.WriteLine($"Single line:{Environment.NewLine}{singleLine}");
Console.WriteLine($"Multiline:{Environment.NewLine}{multiLine}");

Single line:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Proin ac dictum orci, at tincidunt nulla. Donec aliquet, FENNEC eros non interdum posuere, ipsum sapien molestie nunc, nec facilisis libero ipsum et risus. In sed lorem vel ipsum placerat viverra.


Multiline:
[ENTRY NOT FOUND]


In [1]:
var v2Path = Path.Combine(Environment.CurrentDirectory, "data", "v2.cst");
var v2File = File.ReadAllText(v2Path);
var singleLineV2 = CST.Parse(v2File, "Singleline");
var multiLineV2 = CST.Parse(v2File, "Multiline", "DOG", "CAT");;
Console.WriteLine($"Single line v2:{Environment.NewLine}{singleLineV2}");
Console.WriteLine($"Multiline v2:{Environment.NewLine}{multiLineV2}");

Single line v2:
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed ultricies nulla eu tortor mattis, dictum posuere lacus ornare. Maecenas a massa in ligula finibus luctus eu vitae nibh. Proin imperdiet dapibus mauris quis placerat.


Multiline v2:
[ENTRY NOT FOUND]
